<a href="https://colab.research.google.com/github/ttecles/aidl-lyrics-recognition/blob/main/SourceSeparation%2BSpeechRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Colab contains the final model, build from the concatenation of Demucs (for source separation task) and Wav2Vec (for speech recognition task)

Download the libraries needed (Demucs, Transformers[torch])

In [ ]:
!pip install -Uq demucs

     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 194kB 10.5MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 1.9MB 25.9MB/s 


In [ ]:
!pip install transformers[torch]

     |████████████████████████████████| 2.3MB 5.3MB/s 
     |████████████████████████████████| 3.3MB 27.3MB/s 
     |████████████████████████████████| 901kB 39.3MB/s 


Imports

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchaudio
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from IPython.display import Audio, display
import IPython.display as ipd
import soundfile as sf



#Import models:

from demucs.pretrained import load_pretrained
from transformers import AutoTokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor

Download our two models:

In [ ]:
Demucs = load_pretrained("demucs")

Wav2Vec = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
Wav2Vec_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

Downloading: "https://dl.fbaipublicfiles.com/demucs/v3.0/demucs-e07c671f.th" to /root/.cache/torch/hub/checkpoints/demucs-e07c671f.th


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Utility function to change sample rate

In [ ]:
class ChangeSampleRate(nn.Module):
    def __init__(self, input_rate: int, output_rate: int):
        super().__init__()
        self.output_rate = output_rate
        self.input_rate = input_rate

    def forward(self, wav: torch.tensor) -> torch.tensor:
        # Only accepts 1-channel waveform input
        wav = wav.view(wav.size(0), -1)
        new_length = wav.size(-1) * self.output_rate // self.input_rate
        indices = (torch.arange(new_length) * (self.input_rate / self.output_rate))
        round_down = wav[:, indices.long()]
        round_up = wav[:, (indices.long() + 1).clamp(max=wav.size(-1) - 1)]
        output = round_down * (1. - indices.fmod(1.)).unsqueeze(0) + round_up * indices.fmod(1.).unsqueeze(0)
        return output

### **Demucs+Wav2Vec**

In [ ]:
class DemucsWav2Vec(nn.Module):
    def __init__(self):
      super().__init__()
      self.demucs = load_pretrained("demucs")
      self.wav2vec = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
      self.wav2vec_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
      self.changesamplerate = torchaudio.transforms.Resample(44100, 16000)
 
    def forward(self, input_tensor):
 
      #Demucs:
      out_demucs = self.demucs(input_tensor)
 
      #Extract voice and squeeze:
      output_voice = out_demucs[:, 3, :, :]
 
      #transform from stereo to mono:
      output_voice_mono = torch.mean(output_voice, dim=1)
 
      #change sample rate:
      
      output_voice_mono_sr = self.changesamplerate(output_voice_mono)

      #Wav2Vec processor function:
 
      input_values = (output_voice_mono_sr - output_voice_mono_sr.mean(1)) / torch.sqrt(output_voice_mono_sr.var(1) + 1e-5)
 
      #Wav2Vec:
      logits = self.wav2vec(input_values).logits
      predicted_ids = torch.argmax(logits, dim=-1)
      transcription = self.wav2vec_processor.decode(predicted_ids[0])
 
      return transcription

Import random audio mp3 and get input tensor

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
audio, _ = torchaudio.load("/content/gdrive/My Drive/Help2.mp3")
print(type(audio))
sr = 44100 # sample rate
ipd.Audio(audio, rate=sr, autoplay=False)

<class 'torch.Tensor'>


In [ ]:
input_tensor_model = torch.cat([audio.unsqueeze(0)])
print(input_tensor_model.shape)

torch.Size([1, 2, 460800])


Try the model

In [ ]:
Mymodel = DemucsWav2Vec()
transcription = Mymodel(input_tensor_model)
print(transcription)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AND I WAS YOUNG  SO RUCH YOUNGER THAN JODA ON TNEMENE ANY ODIS HALE BEEN N ANYWAY
